In [5]:
import pandas as pd

In [6]:
initial_data = pd.read_csv('initial_data.csv', index_col='id')

initial_cols = ['vendor_id', 'passenger_count', 'pickup_longitude',
                'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
                'trip_duration']

initial_data = initial_data[initial_cols]

In [11]:
initial_data.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,log_trip_duration
id,,,,,,,
id2875421,2,1,-73.982155,40.767937,-73.964630,40.765602,6.122493
id2377394,1,1,-73.980415,40.738564,-73.999481,40.731152,6.498282
id3858529,2,1,-73.979027,40.763939,-74.005333,40.710087,7.661527
id3504673,2,1,-74.010040,40.719971,-74.012268,40.706718,6.063785
id2181028,2,1,-73.973053,40.793209,-73.972923,40.782520,6.077642


In [7]:
#compute MSLE
import numpy as np

initial_data = initial_data.assign(log_trip_duration=np.log1p(initial_data['trip_duration']))
initial_data = initial_data.drop('trip_duration', axis=1)

In [13]:
#Выделим test

from sklearn.model_selection import train_test_split

X = initial_data.drop('log_trip_duration', axis=1)
y = initial_data['log_trip_duration']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [16]:
#Установим метод кросс-валидации KFold

from sklearn.model_selection import KFold

splitter = KFold(n_splits=20, shuffle=True, random_state=33)

In [17]:
#Воспользуемся методом cross_validate

from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

model = LinearRegression()

cv_result = cross_validate(model, X_train, y_train, 
                           scoring='neg_mean_squared_error',
                           cv=splitter, return_train_score=True)

In [18]:
#Возьмем среднее по валидации - качество на кросс-валидации

np.mean(-cv_result['test_score'])

0.6134219105328517

In [19]:
#Финальный замер качества на всей тренировочной выборке
model = LinearRegression()

model.fit(X_train, y_train)

model_predictions = model.predict(X_test)

np.mean(((model_predictions - y_test))**2)

0.6056668879526188

In [20]:
#предобработанные признаки
processed_data = pd.read_csv('processed_data.csv', index_col='id')

In [21]:
processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,
id2875421,1,930.399753,0,455.0,1.500479
id2377394,0,930.399753,0,663.0,1.807119
id3858529,1,930.399753,0,2124.0,6.392080
id3504673,1,930.399753,0,429.0,1.487155
id2181028,1,930.399753,0,435.0,1.189925


In [22]:
import numpy as np

processed_data = processed_data.assign(log_trip_duration=np.log1p(processed_data['trip_duration']))
processed_data = processed_data.drop('trip_duration', axis=1)

In [23]:
X_2 = processed_data.drop('log_trip_duration', axis=1)
y_2 = processed_data['log_trip_duration']

In [24]:
test_indexes = X_test.index
train_indexes = X_train.index

In [25]:
X_train_2 = X_2[X_2.index.isin(train_indexes)]
y_train_2 = y_2[y_2.index.isin(train_indexes)]

X_test_2 = X_2[X_2.index.isin(test_indexes)]
y_test_2 = y_2[y_2.index.isin(test_indexes)]

In [26]:
cv_result_2 = cross_validate(model, X_train_2, y_train_2, 
                             scoring='neg_mean_squared_error',
                             cv=splitter, return_train_score=True)

In [27]:
np.mean(-cv_result_2['test_score'])

0.4313257704232939

In [29]:
#Теперь построим модель и замерим качество на тесте. Модель с предобработанными признаками показывает имеет лучшее качество.

model_2 = LinearRegression()

model_2.fit(X_train_2, y_train_2)

model_predictions_2 = model_2.predict(X_test_2)

np.mean(((model_predictions_2 - y_test_2))**2)

0.4074669546810228